In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup, AdamW

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return

def load_transformer_based_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForMaskedLM.from_pretrained(model_path ,output_hidden_states=True)
    return tokenizer, model

class BERT(nn.Module):
    def __init__(self, model):
        super(BERT, self).__init__()
        self.bert_model = model
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(768, 1)

    def forward(self,ids,mask,token_type_ids):
        o2= self.bert_model(ids,mask,token_type_ids).hidden_states[0]
        o2= o2.max(dim=1).values
        o2 = self.dropout(o2)
        # print('------->>>>>>>>>',o2.values.shape)
        out= self.out(o2)

        return out, o2

class BertDataset(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv[index]

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]

        return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long)


def get_prediction(data, tokenizer, network):
  dataset= BertDataset(data, tokenizer, max_length=200 )
  data_loader=DataLoader(dataset=dataset, shuffle=False)
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device = torch.device(device)
  network = network.to(device)
  network.eval()
  l = []
  emb = []
  for batch in tqdm(data_loader):
    input_ids, token_type_ids, attention_masks = batch
    input_ids = input_ids.to(device)
    token_type_ids = token_type_ids.to(device)
    attention_masks = attention_masks.to(device)

    with torch.no_grad():
        preds, hidden_states = network(input_ids, attention_masks,token_type_ids)
        preds =  torch.where(preds >= 0, 1, 0)
        l.append(preds)
        emb.append(hidden_states)
  return torch.vstack(l).cpu().numpy().tolist(), torch.vstack(emb).cpu().numpy()

In [7]:
model_path = "ka05ar/banglabert-sentiment" #change here........ (for toxicity, use "FredZhang7/one-for-all-toxicity-v3", for sentiment use "ka05ar/banglabert-sentiment", for hatespeech use "Hate-speech-CNERG/bengali-abusive-MuRIL", for SarcaSm use "raquiba/sarcasm-detection-BanglaSARC")
tokenizer, b_model = load_transformer_based_model(model_path)
tokenizer.add_tokens(['<Name>' ,'<Gender>'])
b_model.resize_token_embeddings(len(tokenizer))
model=BERT(b_model)
# load saved_model
saved_model_path = r'/content/drive/MyDrive/#Research/# GB/saved_model/Senti_Original.pth' #change here........
load_model(model, None, PATH = saved_model_path)

Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at ka05ar/banglabert-sentiment and are newly initialized: ['generator_lm_head.bias', 'generator_predictions.LayerNorm.bias', 'generator_predictions.LayerNorm.weight', 'generator_predictions.dense.bias', 'generator_predictions.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded model from /content/drive/MyDrive/#Research/# GB/saved_model/Senti_Original.pth


In [22]:
sentences = ['অপুরভ ভাই অসাম', 'পোলা খোর কখন আসবো']  #change here... make sure to pass the sentences within a list
label, embedding = get_prediction(sentences, tokenizer, model)
print('\nPredicted Labels:',label)

100%|██████████| 2/2 [00:02<00:00,  1.42s/it]


Predicted Labels: [[1], [0]]
